# Projet SDP (Mention IA) — Question 1  
## Explication de type (1-1) avec **Gurobi**

Ce notebook traite **uniquement la Question 1** du sujet : formuler et implémenter un programme d’optimisation linéaire (en nombres entiers) qui **calcule une explication de type (1-1)** pour une comparaison $x \succ y$ si elle existe, et qui **retourne un certificat de non‑existence** sinon.

> Remarque : le solveur utilisé est **Gurobi** via `gurobipy`.


## Partie du sujet — 1. Prologue
Nous nous plaçons dans l’exemple de comparaison entre deux candidats (ici notés `x` et `y`) évalués sur des cours $A\dots G$ avec des poids, et classés par **somme pondérée**.

## Partie du sujet — Données (x et y)  

On encode les notes suivantes (sur 100) et les poids :

- Critères : $A,B,C,D,E,F,G$
- Candidat $x$ : 85, 81, 71, 69, 75, 81, 88  
- Candidat $y$ : 81, 81, 75, 63, 67, 88, 95  
- Poids : 8, 7, 7, 6, 6, 5, 6

On définit la **contribution** de chaque critère $k$ à l’affirmation $x \succ y$ par :
$$
d_k = w_k \,(x_k - y_k)
$$
- $d_k>0$ : critère **pour** $x\succ y$ (ensemble `pros`)  
- $d_k<0$ : critère **contre** $x\succ y$ (ensemble `cons`)  
- $d_k=0$ : critère **neutre**


In [19]:
# Données de l'exemple (x ≻ y)
criteres = ["A","B","C","D","E","F","G"]

notes_x = {"A":85,"B":81,"C":71,"D":69,"E":75,"F":81,"G":88}
notes_y = {"A":81,"B":81,"C":75,"D":63,"E":67,"F":88,"G":95}

poids = {"A":8,"B":7,"C":7,"D":6,"E":6,"F":5,"G":6}

# Contributions d_k = w_k (x_k - y_k)
d = {k: poids[k]*(notes_x[k]-notes_y[k]) for k in criteres}
d


{'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}

## Partie du sujet — 2. Formulation : ensembles pros/cons/neutres
On calcule automatiquement les ensembles `pros(x,y)`, `cons(x,y)` et `neutral(x,y)` à partir des contributions $d_k$.

In [20]:
pros = [k for k in criteres if d[k] > 0]
cons = [k for k in criteres if d[k] < 0]
neutral = [k for k in criteres if d[k] == 0]

print("Contributions d_k :")
for k in criteres:
    print(f"  {k} : {d[k]:+d}")

print("\npros(x,y)   =", pros)
print("cons(x,y)   =", cons)
print("neutral(x,y)=", neutral)


Contributions d_k :
  A : +32
  B : +0
  C : -28
  D : +36
  E : +48
  F : -35
  G : -42

pros(x,y)   = ['A', 'D', 'E']
cons(x,y)   = ['C', 'F', 'G']
neutral(x,y)= ['B']


## Partie du sujet — Trade-offs (1-1)

Un trade-off de type **(1-1)** est une paire $(P,C)$ avec :
- $P \in \mathrm{pros}(x,y)$
- $C \in \mathrm{cons}(x,y)$
- et $d_P + d_C > 0$

On peut donc lister toutes les paires admissibles $(P,C)$ par simple énumération.


In [21]:
tradeoffs_11 = []
for p in pros:
    for c in cons:
        if d[p] + d[c] > 0:
            tradeoffs_11.append((p,c,d[p]+d[c]))

print("Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :")
for (p,c,val) in sorted(tradeoffs_11, key=lambda t: (-t[2], t[0], t[1])):
    print(f"  ({p},{c}) : d_{p}+d_{c} = {val:+d}")

Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :
  (E,C) : d_E+d_C = +20
  (E,F) : d_E+d_F = +13
  (D,C) : d_D+d_C = +8
  (E,G) : d_E+d_G = +6
  (A,C) : d_A+d_C = +4
  (D,F) : d_D+d_F = +1


## Partie du sujet — Question 1 : formulation par optimisation (PLNE)

### Variables de décision
Pour chaque paire admissible $(p,c)$, on introduit une variable binaire :
$$
x_{p,c} \in \{0,1\}
$$
avec $x_{p,c}=1$ si on retient le trade-off $(p,c)$.

### Contraintes
1) **Chaque critère contre** doit être compensé exactement une fois :
$$
\forall c \in \mathrm{cons}(x,y),\quad \sum_{p \in \mathrm{pros}(x,y)} x_{p,c} = 1
$$

2) **Disjonction** : un même critère pour ne peut servir qu’une fois :
$$
\forall p \in \mathrm{pros}(x,y),\quad \sum_{c \in \mathrm{cons}(x,y)} x_{p,c} \le 1
$$

3) Variables définies **uniquement** pour les paires admissibles $(p,c)$ où $d_p + d_c > 0$.

### Objectif
On peut minimiser le nombre de paires sélectionnées :
$$
\min \sum_{(p,c)} x_{p,c}
$$
(avec les contraintes ci-dessus, la solution—si elle existe—utilise en pratique $|\mathrm{cons}(x,y)|$ paires, mais garder un objectif rend la formulation standard.)

### Certificat de non-existence
Si le modèle est **infaisable**, on demande à Gurobi un **IIS** (*Irreducible Inconsistent Subsystem*) : un sous-ensemble minimal de contraintes incompatible. C’est un **certificat de non‑existence** exploitable dans un rendu.


In [22]:
# Implémentation avec Gurobi
# - Si le modèle est faisable : on extrait une explication (1-1)
# - Sinon : on calcule un IIS comme certificat de non-existence

try:
    import gurobipy as gp
    from gurobipy import GRB
except Exception as e:
    raise ImportError(
        "Impossible d'importer gurobipy.\n"
        "Vérifiez que Gurobi est installé et qu'une licence est disponible.\n"
        f"Détail: {e}"
    )

# Construction de l'ensemble des paires admissibles
A = [(p,c) for p in pros for c in cons if d[p] + d[c] > 0]

m = gp.Model("explication_1_1")

# Variables binaires x[p,c] pour (p,c) admissible
xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

# Chaque 'contre' doit être couvert exactement une fois
c_cover = {}
for c in cons:
    c_cover[c] = m.addConstr(gp.quicksum(xvar[p,c] for p in pros if (p,c) in xvar) == 1,
                             name=f"couvrir_cons[{c}]")

# Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
p_use = {}
for p in pros:
    p_use[p] = m.addConstr(gp.quicksum(xvar[p,c] for c in cons if (p,c) in xvar) <= 1,
                           name=f"utiliser_pros_au_plus_une_fois[{p}]")

# Objectif : minimiser le nombre de paires sélectionnées
m.setObjective(gp.quicksum(xvar[p,c] for (p,c) in A), GRB.MINIMIZE)

m.optimize()

if m.Status == GRB.OPTIMAL:
    print("\n Une explication (1-1) existe. Paires sélectionnées :\n")
    solution = [(p,c) for (p,c) in A if xvar[p,c].X > 0.5]
    for (p,c) in solution:
        print(f"  - ({p},{c})  avec d_{p}={d[p]:+d} et d_{c}={d[c]:+d}  => somme={d[p]+d[c]:+d}")
    print(f"\nLongueur ℓ = {len(solution)}")

elif m.Status == GRB.INFEASIBLE:
    print("\n Aucune explication (1-1) n'existe : modèle infaisable.")
    print("Calcul d'un IIS (certificat de non-existence) ...")
    m.computeIIS()
    print("\nContraintes appartenant à l'IIS :")
    for constr in m.getConstrs():
        if constr.IISConstr:
            print(" -", constr.ConstrName)
else:
    print("\nStatut inattendu :", m.Status)


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]


Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x0c593c2c
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%

 Une explication (1-1) existe. Paires sélectionnées :

  - (A,C)  avec d_A=+32 et d_C=-28  => somme=+4
  - (D,F)  avec d_D=+36 et d_F=-35  => somme=+1
  - (E,G)  avec d_E=+48 et d_G=-42  => somme=+6

Longueur ℓ = 3


## Interprétation en langage naturel (sortie attendue)

Quand une solution est trouvée, on peut la transcrire en arguments du type :

- « l’avantage de $x$ sur $y$ en $P$ pèse plus fort que son désavantage en $C$ »

où $(P,C)$ est une paire sélectionnée et $d_P + d_C > 0$.

Le code ci-dessus affiche directement ces paires et les valeurs $d_P$, $d_C$ et $d_P+d_C$.
